In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import nibabel as nib
import os

In [ ]:
raw_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/raw_data/3D_BRAIN_NOT_NORMALIZED/test/ad/I297850.nii.gz"
patient_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial/test/ad/I300337.nii.gz"
patient = nib.load(raw_path)

atlas_path = "HarvardOxford-cort_and_sub-maxprob-thr25-1mm.nii.gz.nii"
atlas_big = nib.load(atlas_path)

##### ALINHAMENTO DA MÁSCARA COM MRI

In [ ]:
data_atlas = atlas_big.get_fdata()[17:162, 19:201, 0:155]
data_patient = patient.get_fdata()
print(patient.shape)
print(data_atlas.shape)

lim = data_atlas.shape[2]

slices = (60, 68, 76, 82, 89, 92, 96)

data_atlas_slices = data_atlas[:, :, slices]
data_patient_slices = data_patient[:, :, slices]

for z in slices:
    atlas_slice = data_atlas[:, :, z]
    patient_slice = data_patient[:, :, z]

    atlas_slice = np.rot90(atlas_slice, k=2)
    patient_slice = np.rot90(patient_slice, k=2)

    plt.figure(figsize=(6,6))
    plt.imshow(patient_slice, cmap='gray')
    plt.imshow(atlas_slice, cmap='tab20', alpha=0.3)  # Ajuste alpha para mais/menos transparência

    plt.title(f"z = {z}")
    plt.axis("on") 
    plt.show()

##### PLOT MÁSCARA COMPLETA

In [ ]:
patient_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial/test/ad/I300337.nii.gz"
patient_pred = nib.load(patient_path).get_fdata()

# Pegar intervalos
intervals = {
    "Ventrículos": range(0, 12),
    "Córtex e outras regiões": range(101, 149)  
}

# Máscara por intervalo
for group_name, label_range in intervals.items():
    fig, axes = plt.subplots(1, 7, figsize=(20, 5))
    patient_mask = data_patient_slices.copy()
    i = 0

    for z in range(0, 7):
        atlas_slice = data_atlas_slices[:, :, z]
        patient_slice = patient_pred[:, :, z]

        # Exibir as imagens lado a lado
        axes[i].imshow(patient_slice, cmap='gray')
        axes[i].imshow(atlas_slice, cmap='jet', alpha=0.5) # o alpha é a transparência
        axes[i].set_title(f"z = {z}")
        axes[i].axis("off")
        i += 1 

    plt.suptitle(f"Grupo: {group_name}", fontsize=16)
    plt.subplots_adjust(top=1.3)  # Ajusta o espaço entre título e imagens
    plt.show()

##### PLOT MÁSCARA VENTRÍCULOS X CÓRTEX

In [ ]:
mri_masked_dict = {}

# Máscara por intervalo
for group_name, label_range in intervals.items():
    fig, axes = plt.subplots(1, 7, figsize=(20, 5))
    patient_mask = data_patient_slices.copy()
    i = 0

    for z in range(0, 7):
        atlas_slice = data_atlas_slices[:, :, z]
        patient_slice = patient_pred[:, :, z]

        mask = np.isin(atlas_slice, list(label_range)).astype(np.uint8) 

        # Aplicar máscara à MRI
        patient_mask[:, :, z] = patient_slice * mask

        # Exibir as imagens lado a lado
        axes[i].imshow(patient_slice, cmap='gray')
        axes[i].imshow(mask, cmap='jet', alpha=0.5) # o alpha é a transparência
        axes[i].set_title(f"z = {slices[z]}")
        axes[i].axis("off")
        i += 1

    mri_masked_dict[group_name] = patient_mask 

    plt.suptitle(f"Grupo: {group_name}", fontsize=16)
    plt.subplots_adjust(top=1.3)  # Ajusta o espaço entre título e imagens
    plt.show()

In [ ]:
# Definir cores para cada região
color_map = {
    "Ventrículos": [1, 0, 0],  # Vermelho
    "Córtex e outras regiões": [0, 0, 1],  # Azul
}

for group_name, label_range in intervals.items():
    fig, axes = plt.subplots(1, 7, figsize=(20, 5))

    for i, z in enumerate(range(7)):
        atlas_slice = data_atlas_slices[:, :, z]  # Máscara segmentada
        patient_slice = data_patient_slices[:, :, z]  # Imagem original MRI

        # Criar máscara binária para a região selecionada
        mask = np.isin(atlas_slice, list(label_range)).astype(np.uint8)

        # Criar máscara para evitar alterar o fundo
        background_mask = patient_slice > 0  # True para pixels do cérebro, False para o fundo preto

        # Criar imagem RGBA (com 4 canais: R, G, B, Alpha)
        mask_rgba = np.zeros((*mask.shape, 4))  # Inicializa com fundo transparente

        # Aplicar cor apenas nas regiões do cérebro, mantendo o fundo preto sem alteração
        mask_rgba[(mask == 0) & background_mask, :3] = color_map[group_name]  # Define a cor (R, G, B)
        mask_rgba[(mask == 0) & background_mask, 3] = 1.0  # Alpha = 1 (opaco) apenas nas regiões válidas

        # Exibir a imagem original em escala de cinza
        axes[i].imshow(patient_slice, cmap="gray")

        # Sobrepor a máscara (sem transparência nas regiões cobertas)
        axes[i].imshow(mask_rgba)  

        axes[i].set_title(f"z = {z}")
        axes[i].axis("off")

    plt.suptitle(f"Grupo: {group_name}", fontsize=16)
    plt.subplots_adjust(top=1.3)
    plt.show()

##### VISUALIZAR OCLUSÃO

In [ ]:
print(mri_masked_dict.keys())

#print(mri_masked_dict['Cérebro completo'].shape)
print(mri_masked_dict['Córtex e outras regiões'].shape)
print(mri_masked_dict['Ventrículos'].shape)

print(data_patient_slices.shape)

for item in mri_masked_dict:
    plt.imshow(mri_masked_dict[item][:, :, 0], cmap='gray')
    plt.title(item)
    plt.show()

##### PREDIZER OCLUSÃO

In [ ]:
model_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/processed_7_slices_data/7_slices_axial/results/3d/axial_certo/test_5/binary_classifier_120_epochs_batch_64_5_classes.keras"
model = tf.keras.models.load_model(model_path)

# Suponha que `data_patient` seja o MRI original
original_image = np.expand_dims(patient_pred, axis=(0, -1))  # Adiciona batch e canal (1, H, W, D, 1)

# Fazer a predição na imagem original
original_pred = model.predict(original_image)[0]
print(original_pred)

# Armazenar resultados
importance_map = {}

# Iterar sobre cada versão ocluída do dicionário
for region_name, occluded_mri in mri_masked_dict.items():
    print(region_name)
    occluded_image = np.expand_dims(occluded_mri, axis=(0, -1))  # Adicionar batch e canal

    # Fazer predição na versão ocluída
    occluded_pred = model.predict(occluded_image)[0]
    print(occluded_pred)

    # Calcular a diferença na predição
    importance = np.abs(original_pred - occluded_pred)

    # Armazenar a importância da região
    importance_map[region_name] = importance

    #print(f"Região: {region_name} → Impacto na predição: {importance}")

print(f"IMPORTANCIA:\n")
for item in mri_masked_dict:
    print(f"{item}: {importance_map[item]}")